https://booking.uz.gov.ua/mobile/train_search/station/?term=З.п.669+Км

POST /mobile/train_search/ HTTP/1.1
Host: booking.uz.gov.ua
Content-Type: application/x-www-form-urlencoded
Cache-Control: no-cache
Postman-Token: 56813cdf-9214-dfd1-4d1f-fc58dc4b19b8

date=2017-12-27&from=2200001&time=00%3A00&to=2204001&get_tpl=1

POST /mobile/train_wagons/ HTTP/1.1
Host: booking.uz.gov.ua
Content-Type: application/x-www-form-urlencoded
Cache-Control: no-cache
Postman-Token: ced0c0d4-6f77-6a30-5f2e-6ce7dc3b47b3

date=2017-12-27&from=2200001&to=2204001&train=064%D0%9A

In [ ]:
import certifi
import urllib3
import urllib.parse
import csv
import json
import re
import pandas as pd
import numpy as np

In [ ]:
search_station_url = 'https://booking.uz.gov.ua/mobile/train_search/station/?term='
alphabet = 'абвгґдеєжзийклмніїопрстуфхцчшщьюяъыэё'

In [ ]:
df_stations_sched = pd.read_csv('stations_all.csv')
df_stations_sched['name'].replace('Ім.тараса  Шевченка', 'Ім. Тараса  Шевченка', inplace=True)
df_stations_sched['name'].replace('Просяна Ім.приклонського В.в.', 
                                  'Просяна Ім. Приклонського В. В.', inplace=True)
df_stations_sched['name'].replace('Ім.бориса Олійника', 'Ім. Бориса Олійника', inplace=True)
df_stations_sched['name'].replace('Ім.олега Крючкова', 'Ім. Олега Крючкова', inplace=True)

In [ ]:
def getBookingStationInfo(station_name):
    #http = urllib3.PoolManager()
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    station_url_full = search_station_url+urllib.parse.quote(station_name, safe='')
    #print('GET url: '+station_url_full)
    request = http.request('GET', station_url_full)
    json_data = request.data.decode('utf-8')
    #print(json_data) 
    loaded_json = json.loads(json_data)
    return loaded_json

In [ ]:
#print(getBookingStationInfo(all_stations[0][1]))
#print(getBookingStationInfo('Гали'))

In [ ]:
for char1 in alphabet:
    for char2 in alphabet:
        for char3 in alphabet:
            for char4 in alphabet:
                for char5 in alphabet:
                    token = char1+char2+char3+char4+char5
                    if token.

In [ ]:
print(len(char_pairs))

In [ ]:
booking_stations_list = []
for index_sched, row_sched in df_stations_sched.iterrows():
    token = row_sched['name']
    book_stations = getBookingStationInfo(token)
    if (len(book_stations) == 0):
        token = token[:-1]
        book_stations = getBookingStationInfo(token)
    if (len(book_stations) == 0):
        token = token[:-2]
        book_stations = getBookingStationInfo(token)
    if (len(book_stations) > 0):
        for book_station in book_stations:
            station_id = book_station['value']
            station_title = book_station['title']
            station_region = book_station['region']
            station = {"id":station_id, "name":station_title, "region":station_region}
            if station not in booking_stations_list:
                booking_stations_list.append(station)
    else:
        print('No booking stations for name\''+ row_sched['name'] +'\' id:'+ str(row_sched['id']))
        
df_booking_stations = pd.DataFrame(booking_stations_list)
df_booking_stations.to_csv('booking_stations_from_sched.csv', index=False, encoding='utf-8')

In [ ]:
all_booking_stations = {}
for char_pair in char_pairs:
    booking_stations = getBookingStationInfo(char_pair)
    if (len(booking_stations) > 0):
        booking_stations_list = []
        for booking_station in booking_stations:
            booking_station_id = booking_station['value']
            booking_station_title = booking_station['title']
            if booking_station_id not in all_booking_stations:
                all_booking_stations[booking_station_id] = booking_station_title
    else:
        print('No booking stations for token \''+ char_pair +'\'')

In [ ]:
csv_file_name = 'booking_stations_all_2.csv'
with open(csv_file_name, 'w', newline='', encoding = "utf-8") as csv_file:        
    writer = csv.writer(csv_file)
    for key in all_booking_stations:
        writer.writerow([key, all_booking_stations[key]])
    print('All stations table saved to file: '+csv_file_name)